# 1. Variables for Inclusive Ratings
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
We obtain the variables needed in equation 1

In [1]:
import psycopg2 
import pandas as pd
from sqlalchemy import create_engine
import ast

In [7]:
# We carry out the connection to the DB
credentials = ast.literal_eval(open(r"C:\Users\pvilas\OneDrive - unizar.es\Python Investigación\credentials.txt", "r").read())
conn = psycopg2.connect(dbname='ESGRefinitiv', user=credentials['psycopg2'][0], 
                        password=credentials['psycopg2'][1], host=credentials['psycopg2'][2], port =credentials['psycopg2'][3])
cur = conn.cursor()

<body> <p  style = "font-family:georgia,garamond,serif;font-size:22px;text-align:justify"></body>
<b>a11_monthly_esg_clean</b></p>
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
The scores of this TABLE complies the following restrictitions: 1. Dates: between BDATE and TIME. 2 Not null vaules(NaN) in the overall and each pillar score. We add some information about country and industry.
<br><b>Tables used:</b><br>
monthly_esg <br>
companies_esg_refinitiv <br>
company_country <br>
classification_ds_country <br>
classification_icb_sector <br>
classification_iso_country <br>

In [4]:
# A11 A11 A11
cur.execute("""
DROP TABLE IF EXISTS
    a11_monthly_esg_clean;
CREATE TABLE
    a11_monthly_esg_clean AS
SELECT 
    info.internal_id, esg.monthly_date, esg.esg_score, esg.environment_pillar_score, esg.social_pillar_score,
    esg.governance_pillar_score, icb.industry_name as industry,  icb.sector_name as sector, 
    ds_country.country_name as ds_country, ds_country.country_iso_id, iso.continent
FROM 
    monthly_esg AS esg
    INNER JOIN companies_esg_refinitiv AS info
    ON esg.datastream_id = info.datastream_id
    INNER JOIN company_country AS country
    ON info.datastream_id = country.datastream_id
    INNER JOIN classification_ds_country AS ds_country
    ON country.ds_country_group_id = ds_country.country_ds_id    
    INNER JOIN classification_icb_sector AS icb
    ON info.icb_subsector_id = icb.subsector_id
    INNER JOIN classification_iso_country AS iso
    ON ds_country.country_iso_id= iso.country_iso_id
WHERE 
    esg.monthly_date BETWEEN info.ds_first_value AND info.ds_last_value 
    AND (esg.esg_score,  esg.environment_pillar_score, esg.social_pillar_score,
    esg.governance_pillar_score ) IS NOT NULL
    """ )


conn.commit()

<body> <p  style = "font-family:georgia,garamond,serif;font-size:22px;text-align:justify"></body>
<b>a12_daily_mv_clean</b></p>
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
    The market value of this TABLE complies the following restrictions: 1 Dates: between BDATE and TIME. 2 Not null vaules(NaN) in MV column. We also get the total index return in USD dollars. 
<br><b> Tables used:</b><br>
daily_market <br>
companies_esg_refinitiv <br>
classification_iso_currency  <br>
exchange_rates <br>

In [8]:
cur.execute("""
DROP TABLE IF EXISTS
    a12_daily_mv_clean;
CREATE TABLE 
    a12_daily_mv_clean AS
SELECT
    info.internal_id, market.daily_date, market.total_return_index, market.datastream_id,
    exchange_rates.exchange_rate, market.price_index, 
    market.ds_currency_symbol_id, market.total_return_index * exchange_rates.exchange_rate AS return_index_usd, CASE
        WHEN market.mvc_usd > market.market_value_usd THEN market.mvc_usd
        ELSE market.market_value_usd 
        END AS market_value_usd 
FROM 
    daily_market AS market
    INNER JOIN companies_esg_refinitiv AS info
    ON market.datastream_id = info.datastream_id
    INNER JOIN classification_iso_currency 
    ON classification_iso_currency.ds_currency_symbol_id = market.ds_currency_symbol_id
    INNER JOIN exchange_rates
    ON classification_iso_currency.ds_currency_serie_usd_id = exchange_rates.ds_currency_serie_id AND 
    exchange_rates.daily_date_id = market.daily_date
WHERE 
    market.daily_date BETWEEN info.ds_first_value AND info.ds_last_value 
    AND market.market_value_usd IS NOT NULL 

    """ )
conn.commit()

<body> <p  style = "font-family:georgia,garamond,serif;font-size:22px;text-align:justify"></body>
<b>a21_yearly_esg_mv</b></p>
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
Yearly average of each score stored in the TABLE <b>a11_monthly_esg_clean</b> and the yearly average of the market value in USD stored in the Table <b>a12_daily_mv_clean</b>.
<b>Tables used:</b><br>
a11_monthly_esg_clean <br>
a12_daily_mv_clean <br>

In [7]:
# A21 A21 A21
cur.execute("""
DROP TABLE IF EXISTS
    a21_yearly_esg_mv;
CREATE TABLE
    a21_yearly_esg_mv AS
SELECT 
    market.internal_id, EXTRACT(YEAR from market.daily_date) AS year, 
    AVG(market.market_value_usd)::NUMERIC(50,10) AS mv_usd_average,     
    LN(AVG(market.market_value_usd))::NUMERIC(50,10) AS natural_log_mv_usd,
    AVG(esg.esg_score)::NUMERIC(50,10) AS esg_score_average, 
    AVG(esg.environment_pillar_score)::NUMERIC(50,10) as environment_pillar_average,
    AVG(esg.social_pillar_score)::NUMERIC(50,10) as social_pillar_average, 
    AVG(esg.governance_pillar_score)::NUMERIC(50,10) AS governance_pillar_average, 
    esg.industry, esg.sector, esg.ds_country, esg.country_iso_id, esg.continent
FROM 
    a12_daily_mv_clean AS market
    INNER JOIN a11_monthly_esg_clean AS esg
    ON market.internal_id = esg.internal_id AND  EXTRACT(YEAR from market.daily_date) = EXTRACT(YEAR from esg.monthly_date)
GROUP BY 
    market.internal_id, EXTRACT(YEAR from market.daily_date), esg.ds_country,  
    esg.industry, esg.sector, esg.country_iso_id, esg.continent
""")
conn.commit()

<body> <p  style = "font-family:georgia,garamond,serif;font-size:22px;text-align:justify"></body>
<b>a22_yearly_esg_mv_percentiles</b></p>
<body> <p  style = "font-family:georgia,garamond,serif;font-size:16px;text-align:justify"></body>
Idem as TABLE <b>a21_yearly_esg_mv</b>. We add a column that returns the yearly percentile rank of the mv_usd_average by year. <br>
<b>Tables used:</b><br> 
a21_yearly_esg_mv : year

In [8]:
# A22 A22 A22
cur.execute("""
DROP TABLE IF EXISTS
    a22_yearly_esg_mv_percentiles;
CREATE TABLE
    a22_yearly_esg_mv_percentiles AS
SELECT 
    year.internal_id, year.year, year.mv_usd_average, year.natural_log_mv_usd, 
    PERCENT_RANK() OVER(PARTITION BY (year) ORDER BY year.mv_usd_average) AS percent_rank_mv_usd,
    year.esg_score_average, year.environment_pillar_average, year.social_pillar_average,
    year.governance_pillar_average, year.industry, year.sector, year.ds_country, year.country_iso_id, 
    year.continent
FROM 
    a21_yearly_esg_mv AS year


""")
# We multiply the columns of the percentile rank by 100
cur.execute(""" 
    UPDATE a22_yearly_esg_mv_percentiles 
    SET percent_rank_mv_usd=percent_rank_mv_usd*100
""")

conn.commit()